# illustration of implement task - draft - reference for discussion

term names order may be slighly different from cuurent architectures which shall be explained in next round
 
architecture.drawio (1).svg

# import data set

In [1]:
!pip install wget
import wget
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
file=wget.download(url)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=6b43256978af9ca7611d61e2aa337c064b612942ab017c122ad3ac2137d8f127
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read() # should be simple plain text file

print('corpus length:', len(text)) # length of the dataset in characters

corpus length: 1115394


In [ ]:
print(text[:500]) # print first 100 characters


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


# methods

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters

# hyperparameters
batch_size = 128 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 4.052276128181501e-03
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 16
n_layer = 8
dropout = 0.25
# ------------
torch.manual_seed(1337)


# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False) # emulates the behaviour : what am i looking for
        self.query = nn.Linear(n_embd, head_size, bias=False) # emulates the behaviour : what do i contain?
        self.value = nn.Linear(n_embd, head_size, bias=False) # emulates the behaviour : the information i can communicate
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # used when the tensor needs to be updated or to store pre computed value , but its neither a parameter or hyperparamter
        
        self.dropout = nn.Dropout(dropout) # regularization technique

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities") . Higher the value , higher the info(relation between what am i looking for and what do i contain)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        # with out normalization , the value of variance of wei would close to C , and then aggregation would create a set of vectors with one value being very high compared to others.

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T) # smart way of averaging out the values per row
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C) 
        # if affinity high ,and the info i can communicate is high , then the result is self attention i.e. given a character,tend to the right set of characters with in a block 
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    # if token is to represented by a embedding of size (1*30) when passed through mullti head attention having 6 heads.
    # then each head will output (1*5) vector. then they are all concatenated to form (1*5) vector
    # basic idea is to allow each head to learn as much info as possible, and if one head dies, it still gets info from other heads  
    # 
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) 
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # output of self attention
        out = self.dropout(self.proj(out)) # apply linear layer output of self attention
        return out

class FeedFoward(nn.Module):
    """ refered to as position - wise feed forward networks in paper"""
    """ a simple linear layer followed by a non-linearity """
   # in simple words now you just 2 layered deep neural network where the first layer is passed through relu , the next through soft max ,
   # where the input is output of multi-headed masled attention """
   # the idea of "4 times" is just copied from paper    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # emulating self.proj
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication (done using multihead attention) followed by computation (feedforward) """
    ### the computations perfromed by taking a vector input(representing char info+pos info) and producing a vector output (output of feedforward)
    ### layer norm done first , then transformation
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) 
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
      # implementations of reisual connection (adding) - to adress isue rnn has
            # not first layer norm done then tranformation done
        x = x + self.sa(self.ln1(x)) # add input to output of multihead attention after passing through layernorm where residual connection happend through ext linear layer
        x = x + self.ffwd(self.ln2(x))# add input to output of feedforward after passing through layernorm where residual connection happened through self.proj 
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):
    
    # brief explanation of the forward pass :
    # 1. represent each token usng vector of size n_embed using nn.embedding
    # 2. position of each token in a block can be represented using sorted vector of integers of block size 
    # 3. represent it in the same size of 1. .
    # 4. add both vectors
    # 5. pass through linear layer which converts it back to the vocab size
    # 6. now each token encodes the information of about itself and its position 

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table , same as linear layer, but ascess non zero elements using indices
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None): # targets= None, for forward pass , first time
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
      
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens to handle the situation that if idx is at last n character of doc , the tokens dont thinnk about access element out of doc . 
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)



# training

In [ ]:
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

6.362177 M parameters
step 0: train loss 2.0569, val loss 2.1248
step 100: train loss 1.9146, val loss 2.0324
step 200: train loss 1.7860, val loss 1.9465
step 300: train loss 1.6894, val loss 1.8647
step 400: train loss 1.6192, val loss 1.7903
step 500: train loss 1.5635, val loss 1.7430
step 600: train loss 1.5198, val loss 1.7166
step 700: train loss 1.4784, val loss 1.6859
step 800: train loss 1.4538, val loss 1.6608
step 900: train loss 1.4313, val loss 1.6284
step 1000: train loss 1.4091, val loss 1.6252
step 1100: train loss 1.3930, val loss 1.6147
step 1200: train loss 1.3746, val loss 1.5874
step 1300: train loss 1.3603, val loss 1.5846
step 1400: train loss 1.3462, val loss 1.5755
step 1500: train loss 1.3335, val loss 1.5695
step 1600: train loss 1.3301, val loss 1.5570
step 1700: train loss 1.3193, val loss 1.5580
step 1800: train loss 1.3167, val loss 1.5566
step 1900: train loss 1.3026, val loss 1.5427
step 2000: train loss 1.2960, val loss 1.5325
step 2100: train loss 1.

# tuning

In [ ]:
!pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.7 MB/s eta 0:00:00


In [ ]:
max_iters = 2000


In [ ]:
import optuna
import torch
import torch.nn as nn
from torch.nn import functional as F


def objective(trial):

    # suggest values for hyperparameters to optimize
    batch_size = trial.suggest_categorical('batch_size', [64, 128,256]) # 2 power idea
    block_size = trial.suggest_categorical('block_size', [64,128,256,512]) # 2 power idea
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-4, log=True) # dint exceed 1e-3 , because of self attention behaviour
    n_embd = trial.suggest_categorical('n_embd', [125,200,250,300,350,400,450,500]) # 2 power idea
    n_head = trial.suggest_categorical('n_head', [8,16,32])
    n_layer = trial.suggest_int('n_layer', 4, 8)
    dropout = trial.suggest_float('dropout', 0.0, 0.1)
    flcr=trial.suggest_int('flcr', 3, 8)
    
    # define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # used adam optimizer
    loss_fn = nn.CrossEntropyLoss()

    # train the model for a few iterations and estimate the validation loss
    for i in range(max_iters):
        # get a batch of data and compute the forward pass
        X, Y = get_batch('train')
        logits, loss = model(X, Y)
        
        # compute gradients and update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # evaluate the model on validation data and estimate the loss
        if i % eval_interval == 0:
            losses = estimate_loss()
            val_loss = losses['val']

            # report intermediate results to optuna
            trial.report(val_loss, i)

            # early stopping based on validation loss
            if i > eval_interval and val_loss > max(losses['val'] for i in range(i-eval_interval, i)):
                break

    # return the validation loss as the objective value to minimize
    return val_loss

# create a study object and run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)


[I 2023-04-07 03:54:52,505] A new study created in memory with name: no-name-5a60d868-dec0-4d66-ac40-956f4ff4f193
[I 2023-04-07 03:59:49,787] Trial 0 finished with value: 1.7463330030441284 and parameters: {'batch_size': 256, 'block_size': 512, 'learning_rate': 1.538034490090522e-05, 'n_embd': 500, 'n_head': 16, 'n_layer': 6, 'dropout': 0.009603068747981836, 'flcr': 3}. Best is trial 0 with value: 1.7463330030441284.
[I 2023-04-07 04:04:46,880] Trial 1 finished with value: 1.6952303647994995 and parameters: {'batch_size': 256, 'block_size': 512, 'learning_rate': 2.2955783196330685e-05, 'n_embd': 200, 'n_head': 8, 'n_layer': 4, 'dropout': 0.09690449517594718, 'flcr': 4}. Best is trial 1 with value: 1.6952303647994995.
[I 2023-04-07 04:09:44,161] Trial 2 finished with value: 1.6470855474472046 and parameters: {'batch_size': 128, 'block_size': 64, 'learning_rate': 2.950342581562844e-05, 'n_embd': 200, 'n_head': 16, 'n_layer': 7, 'dropout': 0.009943376026114937, 'flcr': 7}. Best is trial 2

In [ ]:
print("Optimal hyperparameters:")
print(f"Batch size: {study.best_params['batch_size']}")
print(f"Block size: {study.best_params['block_size']}")
print(f"Learning rate: {study.best_params['learning_rate']}")
print(f"n_embd: {study.best_params['n_embd']}")
print(f"n_head: {study.best_params['n_head']}")
print(f"n_layer: {study.best_params['n_layer']}")
print(f"dropout: {study.best_params['dropout']}")
#print(f"optimizer: {study.best_params['optimizer']}")

Optimal hyperparameters:
Batch size: 128
Block size: 128
Learning rate: 3.5381318961517773e-05
n_embd: 250
n_head: 16
n_layer: 8
dropout: 0.0797896671079949


In [ ]:
# Save the trained model's weights to a file
torch.save(model.state_dict(), "model_weights.pth")

In [ ]:
import json

# Define hyperparameters as a dictionary
hyperparameters = {
   "batch_size": study.best_params['batch_size'],
    "block_size": study.best_params['block_size'],
    "learning_rate": study.best_params['learning_rate'],
    "n_embd": study.best_params['n_embd'],
    "n_head": study.best_params['n_head'],
    "n_layer": study.best_params['n_layer'],
    "dropout": study.best_params['dropout']
}

# Write hyperparameters to a JSON file
with open("hyperparameters.json", "w") as f:
    json.dump(hyperparameters, f)

# to run on saved data

In [ ]:
import json
import torch
#MyModel =  BigramLanguageModel() # replace with the actual name of your model class

# Load hyperparameters from JSON file
with open("hyperparameters.json", "r") as f:
    hyperparameters = json.load(f)

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 4.673588482742393e-05
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout =  0.08887636695128351



# Initialize the model with the loaded hyperparameters
model2 = BigramLanguageModel()

# Load the model weights from the PyTorch file
model2.load_state_dict(torch.load("model_weights.pth"))

# Set the model to evaluation mode
model2.eval()
m=model2.to(device)
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

RuntimeError: ignored

In [ ]:
# Load model weights
model_weights = load_state_dict(torch.load("model_weights.pth"))

# Instantiate model
model = MultiHeadAttention(num_heads=hyperparameters['n_head'],
                           head_size=hyperparameters['n_embd']).to(device)
model.load_state_dict(model_weights)

model.eval()
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

NameError: ignored